In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import shutil

import numpy as np
import pandas as pd

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

#Keras


from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
#from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
df_acc = pd.read_csv('../data/acc_1.csv')
#df_gyr = pd.read_csv('../data/gyro_2.csv')
#df_mag = pd.read_csv('../data/mag_1.csv')

In [3]:
df_clean_acc=df_acc[['x','y','z','train']]
df_clean_acc.train = pd.Categorical(df_clean_acc.train)

In [4]:
df_clean_acc['response']=df_clean_acc.train.cat.codes
df_clean_acc['response']=1-df_clean_acc['response']

In [5]:
X = df_clean_acc[['x','y','z']].values.astype(float) # sensor data
Y = df_clean_acc['response'].values
# Y=Y.reshape(380962,1)

In [6]:
def create_model():
    # Define model
    global model
    model = Sequential()
    model.add(Dense(15, input_dim=3, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
tmodel = create_model()
print(tmodel.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 15)                60        
_________________________________________________________________
dense_2 (Dense)              (None, 15)                240       
_________________________________________________________________
dense_3 (Dense)              (None, 15)                240       
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 32        
Total params: 572
Trainable params: 572
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        
loss_history = LossHistory()
early_stopping = EarlyStopping(monitor='val_acc', patience=20)

In [9]:
from keras.utils.np_utils import to_categorical

estimator = KerasClassifier(create_model, epochs=200, batch_size=100, verbose=False)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=5)

In [10]:
Y_test = to_categorical(Y_test)

In [11]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((266673, 3), (114289, 3), (266673,), (114289, 2))

In [12]:
results = estimator.fit(X_train, Y_train, callbacks=[loss_history, early_stopping], validation_data=(X_test, Y_test))

In [13]:
kfold = KFold(n_splits=10, shuffle=True, random_state=5)

cv_results = cross_val_score(estimator, X_test, Y_test, cv=kfold)

print("Baseline on test data: %.2f%% (%.2f%%)" % (cv_results.mean()*100, cv_results.std()*100))

Baseline on test data: 95.50% (0.32%)


In [19]:
json_model = estimator.model.to_json()
open("../data/hacktrack_arch1.json", "w").write(json_model)

2009

In [20]:
estimator.model.save_weights('../models/hacktrack_weights1.h5', overwrite=True)